In [1]:
import os
from pathlib import Path
from openai import OpenAI, AsyncOpenAI
from dotenv import load_dotenv
from datasets import load_dataset, Dataset

import nest_asyncio
nest_asyncio.apply()

In [2]:
dataset_name = load_dataset(os.getenv("HF_HUB_DATASET_NAME"))
assert dataset_name

In [3]:
# select the first 32 examples
sampled_dataset = dataset_name['train'].select(range(2))
len(sampled_dataset)

2

In [13]:
system_message = """
You are given 5 sentences from a story in a shuffled order. 
Each sentence has a serial number assoiciated with it. 
You can find the serial number at the beginning of each sentence, for example: 1. <sentence>.
Your task is to predict the correct order of sentences which would resemble the original story and then return the serial numbers as a comma separated string.

#IMPORTANT
There are exactly five sentences.
You should only return the serial numbers as a comma separated string.
"""

client = AsyncOpenAI(
    api_key="token-abc123",
    base_url=os.getenv("VLLM_ENDPOINT_URL")
)

In [8]:
from tqdm.auto import tqdm


for _, data in tqdm(enumerate(sampled_dataset)):
    formatted_inputs = []
    for idx, sent in enumerate(data["shuffled_sentences"]):
        formatted_inputs.append( f"{idx + 1}. {sent}")
    

0it [00:00, ?it/s]

In [14]:
response =  await client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": formatted_inputs[0]}
    ],
    temperature=0.0,
)

response

In [12]:
response.choices[0].message.content

"Based on the given sentence, it seems to be a scene from a childhood memory or a scene where the protagonist is young. \n\n2. It was a typical summer day in the small town. \n\nThis sentence doesn't seem to be related to the first sentence, so I'll move on to the next one.\n\n3. Carrie's sister, Emily, had been warned by their parents about watching her little sister.\n\nThis sentence also doesn't seem to be directly related to the first sentence, so I'll move on to the next one.\n\n4. But what Carrie's sister didn't know was that she had a plan to get a ride on her sister's bike.\n\nThis sentence seems to be a continuation of the first sentence, as it explains Carrie's plan to sneak a ride on her sister's bike.\n\n5. So, when Emily got distracted by a group of kids playing baseball, Carrie saw her chance and jumped onto the bike.\n\nThis sentence seems to be a continuation of the previous two sentences, as it shows Carrie's plan being executed.\n\nBased on the analysis, the correct o